# Creación de las vistas materializadas para la capa PLATA

### Vista product
- c01_bronce.Production_Product
- c01_bronce.Production_ProductCategory
- c01_bronce.Production_ProductSubcategory

In [7]:
CREATE MATERIALIZED LAKE VIEW c02_plata.product AS 
SELECT 
    CAST(p.ProductID AS INT) AS ProductID,
    CAST(p.Name AS STRING) AS ProductName,
    CAST(p.ProductNumber AS STRING) AS ProductNumber,
    CAST(p.Color AS STRING) AS Color,
    CAST(p.Size AS STRING) AS Size,
    CAST(p.StandardCost AS DECIMAL(18,2)) AS StandardCost,
    CAST(p.ListPrice AS DECIMAL(18,2)) AS ListPrice,
    CAST(ps.Name AS STRING) AS SubcategoryName,
    CAST(pc.Name AS STRING) AS CategoryName
FROM c01_bronce.Production_Product p
LEFT JOIN c01_bronce.Production_ProductSubcategory ps
    ON p.ProductSubcategoryID = ps.ProductSubcategoryID
LEFT JOIN c01_bronce.Production_ProductCategory pc
    ON ps.ProductCategoryID = pc.ProductCategoryID;

StatementMeta(, 6ff71ceb-838e-411a-8029-2e97a3856ba7, 8, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

### Vista sales_order
- c01_bronce.Sales_SalesOrderHeader
- c01_bronce.Sales_SalesOrderDetail


In [13]:
CREATE MATERIALIZED LAKE VIEW c02_plata.sales_order AS 
WITH header_parsed AS (
  SELECT
    CAST(SalesOrderID AS INT)                                       AS SalesOrderID,
    TO_DATE(TO_TIMESTAMP(OrderDate,'M/d/yyyy h:mm:ss a'))            AS OrderDate,
    TO_DATE(TO_TIMESTAMP(DueDate,'M/d/yyyy h:mm:ss a'))              AS DueDate,
    TO_DATE(TO_TIMESTAMP(ShipDate,'M/d/yyyy h:mm:ss a'))             AS ShipDate,
    CAST(SubTotal AS DECIMAL(18,2))                                  AS SubTotal,
    CAST(TaxAmt AS DECIMAL(18,2))                                    AS TaxAmt,
    CAST(Freight AS DECIMAL(18,2))                                   AS Freight,
    CAST(TotalDue AS DECIMAL(18,2))                                  AS TotalDue,
    CAST(CustomerID AS INT)                                          AS CustomerID,
    CAST(SalesPersonID AS INT)                                       AS SalesPersonID,
    CAST(TerritoryID AS INT)                                         AS TerritoryID,
    CAST(ShipMethodID AS INT)                                        AS ShipMethodID,
    CAST(CurrencyRateID AS INT)                                      AS CurrencyRateID
  FROM c01_bronce.Sales_SalesOrderHeader
)
SELECT
  CAST(d.SalesOrderDetailID AS INT)                                  AS SalesOrderDetailID,
  CAST(d.SalesOrderID AS INT)                                        AS SalesOrderID,
  h.OrderDate,
  h.DueDate,
  h.ShipDate,
  CAST(d.ProductID AS INT)                                           AS ProductID,
  CAST(d.OrderQty AS INT)                                            AS OrderQty,
  CAST(d.UnitPrice AS DECIMAL(18,2))                                 AS UnitPrice,
  CAST(d.UnitPriceDiscount AS DECIMAL(18,2))                         AS UnitPriceDiscount,
  CAST(d.LineTotal AS DECIMAL(18,2))                                 AS LineTotal,
  h.SubTotal,
  h.TaxAmt,
  h.Freight,
  h.TotalDue,
  h.CustomerID,
  h.SalesPersonID,
  h.TerritoryID,
  h.ShipMethodID,
  h.CurrencyRateID
FROM c01_bronce.Sales_SalesOrderDetail d
JOIN header_parsed h
  ON CAST(d.SalesOrderID AS INT) = h.SalesOrderID;

StatementMeta(, 6ff71ceb-838e-411a-8029-2e97a3856ba7, 14, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>